https://www.youtube.com/watch?v=lVXJgn3fDkg

In [1]:
import qiskit
from qiskit import QuantumCircuit, transpile, execute
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor

import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings('ignore')

In [2]:
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

In [3]:
# Display measurement results
def show_barchart(counts):
    y_pos=np.arange(2)
    objects=['a','b']
    plt.xticks(y_pos, objects)
    plt.bar(y_pos, counts)
def show_statevector(statevector):
    print('statevector')
    for z in statevector:
        print(np.around(z,3))
    print('\n')
def get_probabilities(statevector):
    n_uu = np.abs(statevector[0])**2
    n_du = np.abs(statevector[1])**2
    n_ud = np.abs(statevector[2])**2
    n_dd = np.abs(statevector[3])**2
    return n_uu, n_du, n_ud, n_dd


## Verify that the matrix for the gate is unitary

In [4]:
def unitary(U):
    assert(np.around(np.matmul(np.conj(np.transpose(U)),U),5)==np.eye(2)).all()

## Gate for non-vertical measurement basis

The rows of the unitary matrix for the measurement are the complex conjugates of the states of the measurement basis, which is in the yz plane

In [5]:
def measurement_gate(theta, show):
    U=np.array([[math.cos(theta/2),math.sin(theta/2)],\
                [math.sin(theta/2),-math.cos(theta/2)]])
    if show:
        print('unitary matrix for projection')
        print(U[0,0],U[0,1])
        print(U[1,0],U[1,1])
    unitary(U)
    return U
    

## Project state vector onto new measurement basis

In [6]:
def project(bit,circuit, theta, show):
    U = measurement_gate(theta, show)
    circuit.unitary(U,[bit])
    return circuit

## Measure with polarization not vertical

In [7]:
def measure(bit, circuit, angle, show=False):
    circuit = project(bit, circuit, angle, show)
    circuit.measure(bit,bit)
    return circuit

In [8]:
def get_encoding(keybit, basis):
    return 2*keybit+basis

In [9]:
def prepare_states(keybits, bases):
    keysize = len(keybits)
    circuit = qiskit.QuantumCircuit(keysize, keysize)
    for i in range(keysize):
        encoding = get_encoding(keybits[i], bases[i])
        if encoding == 0:
    #             zero degrees
            pass
        elif encoding == 1:
    #             90 degrees
            circuit.h(i)
        elif encoding == 2:
    #             180 degrees
            circuit.x(i)
        else:
    #             270 degrees
            circuit.x(i)
            circuit.h(i)
    return circuit

In [10]:
shots=100
simulate = False
if simulate:
    backend = QasmSimulator()
#     backend = qiskit.Aer.get_backend('statevector_simulator')
else:
#     IBMQ.save_account('efd5018b00323f5067f48f9b32e44c06c925296ffa6ef34f5b21e647925c945a8dd97a62469066d4f53f51f4cfc614ece675ce6a99b4272e636ac4a6faa33e5')

    IBMQ.load_account()
    provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
    for b in provider.backends():
        print(b)
    backend = provider.get_backend('ibmq_quito')    
thetas=math.radians(0), math.radians(90), math.radians(180), math.radians(270)

ibmq_qasm_simulator
ibmq_armonk
ibmq_santiago
ibmq_bogota
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila


## Alice draws a random sequence of key bits

## Alice draws a random sequence of measurement basis vectors

## Bob draws a random sequence of measurement basis vactors

In [11]:
keysize=5
# np.random.seed(0)
alice_key=np.random.randint(2,size=keysize)
bob_key = np.ndarray(keysize, dtype=int)
alice_bases = np.random.randint(2, size=keysize)
bob_bases = np.random.randint(2, size=keysize)

## Alice encodes her choices of key bits and measurement basis vectors into qubits

## Alice transmits her qubits to Bob

## Bob measures Alice's qubits in his chosen measurement bases

## Alice and Bob each publish their chosen measurement bases

In [12]:
circuit = prepare_states(alice_key, alice_bases)
for i, keybit in enumerate(alice_key):
    alice_basis = alice_bases[i]
    bob_basis = bob_bases[i]

    
#         Bob measures Alice's qubit in his chosen measurement basis
    if bob_basis == 0:
        circuit = measure(i, circuit, 0)
    else:
        circuit = measure(i, circuit, math.radians(90))
        
if simulate:
    result = qiskit.execute(circuit, backend=backend).result()
#     statevector = result.get_statevector()
#     show_statevector(statevector)
    compiled_circuit = transpile(circuit)
    job = backend.run(compiled_circuit, shots=shots)
    result = job.result()
else:
#             live backend
    job = execute(circuit, backend=backend, shots=shots)
    job_monitor(job)
    result = job.result()
counts = result.get_counts(circuit)
num_shots_bit_set = np.zeros(keysize, dtype=int)
for key in counts.keys():
    count = counts[key]
    for i in range(keysize):
        if key[keysize-i-1]=='1':
            num_shots_bit_set[i] += count


Job Status: job has successfully run


## Where Alice and Bob chose the same measurement basis, Alice's key bit matches Bob's key bit

In [13]:
for i in range(keysize):
    if alice_bases[i]==bob_bases[i]:
        bob_key = 0
        if num_shots_bit_set[i]>shots/2:
            bob_key = 1
        print(alice_key[i], bob_key)


0 0
1 1
0 0
